In [25]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

# 启用GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 下载并加载数据集
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True,
                                                download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False,
                                               download=True, transform=transforms.ToTensor())
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True,drop_last=True)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=False,drop_last=True)

# 构建模型
w = torch.tensor(np.random.normal(1, 1, (784, 10)), dtype=torch.float32, device=device)
b = torch.zeros(1, dtype=torch.float32, device=device)
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

def lin_reg(X, w, b):
    xsize = X.size()
    return torch.mm(X.reshape(-1, 784), w) + b

# 手动实现softmax
def softmax(X):
    row_max = X.max(axis=1).values
    row_max = row_max.reshape(-1, 1)
    X = X - row_max
    X_exp = torch.exp(X)
    partition = X_exp + torch.ones(X.size(), device=device)
    predict = X_exp / partition
    return predict

# 手动实现交叉熵函数
def entropy_loss(yhat, y):
    loss = 0.0
    for i in range(0, batch_size):
        y_softmax = softmax(yhat)
        index = y[i]
        loss -= y_softmax[i][index].log()
    return loss

def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

lr = 0.01
num_epochs = 20
batch_size = 64

for epoch in range(num_epochs):
    train_size = 0
    for data in train_iter:
        train_size += 1
        instance, label = data[0].to(device), data[1].to(device)
        predict = lin_reg(instance, w, b)
        l = entropy_loss(predict, label)
        l.backward()
        sgd([w, b], lr, batch_size)
        w.grad.data.zero_()
        b.grad.data.zero_()
        
    # 计算loss和准确率
    test_size = len(mnist_test)
    test_loss = 0.0
    correct = 0
    for data in test_iter:
        instance, label = data[0].to(device), data[1].to(device)
        predict = lin_reg(instance, w, b)
        test_loss += entropy_loss(predict, label).item()
        _, predicted_label = torch.max(predict, 1)
        correct += (predicted_label == label).sum().item()
    test_loss /= test_size
    accuracy = correct / test_size
    print('epoch %d, test loss %f, test accuracy %f' % (epoch + 1, test_loss, accuracy))

    #print('epoch %d, loss %f, accuracy %f' % (epoch + 1, epoch_loss.item() / test_size, accuracy))

epoch 1, test loss 3.452282, test accuracy 0.411800
epoch 2, test loss 2.696389, test accuracy 0.521600
epoch 3, test loss 2.378341, test accuracy 0.567500
epoch 4, test loss 2.182095, test accuracy 0.599900
epoch 5, test loss 2.048137, test accuracy 0.617700
epoch 6, test loss 1.957259, test accuracy 0.636700
epoch 7, test loss 1.881521, test accuracy 0.646900
epoch 8, test loss 1.825456, test accuracy 0.655800
epoch 9, test loss 1.781432, test accuracy 0.667300
epoch 10, test loss 1.740455, test accuracy 0.671500
epoch 11, test loss 1.709622, test accuracy 0.679000
epoch 12, test loss 1.683567, test accuracy 0.686900
epoch 13, test loss 1.657495, test accuracy 0.688200
epoch 14, test loss 1.635059, test accuracy 0.691600
epoch 15, test loss 1.614177, test accuracy 0.696900
epoch 16, test loss 1.594583, test accuracy 0.701100
epoch 17, test loss 1.573997, test accuracy 0.703900
epoch 18, test loss 1.557697, test accuracy 0.710000
epoch 19, test loss 1.543478, test accuracy 0.710900
ep

In [22]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# 构建模型
class SoftmaxRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

# 定义参数
input_dim = 784
output_dim = 10
lr = 0.01
num_epochs = 20

# 加载数据集
transform = transforms.ToTensor()
train_dataset = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True,
                                                 download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False,
                                                download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# 启用GPU
model = SoftmaxRegression(input_dim, output_dim)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# 训练模型
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, input_dim).to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 测试
    correct = 0
    total = 0
    epoch_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.reshape(-1, input_dim).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            epoch_loss += criterion(outputs, labels)

    accuracy = correct / total
    print('epoch %d, loss %f, accuracy %f' % (epoch + 1, epoch_loss/len(test_loader), accuracy))

epoch 1, loss 0.739076, accuracy 0.755700
epoch 2, loss 0.646312, accuracy 0.786400
epoch 3, loss 0.604927, accuracy 0.798900
epoch 4, loss 0.579350, accuracy 0.807300
epoch 5, loss 0.561012, accuracy 0.810900
epoch 6, loss 0.546096, accuracy 0.816500
epoch 7, loss 0.535844, accuracy 0.819200
epoch 8, loss 0.527357, accuracy 0.821600
epoch 9, loss 0.520109, accuracy 0.824100
epoch 10, loss 0.516812, accuracy 0.824100
epoch 11, loss 0.510586, accuracy 0.825300
epoch 12, loss 0.504955, accuracy 0.828200
epoch 13, loss 0.505228, accuracy 0.826500
epoch 14, loss 0.498106, accuracy 0.827800
epoch 15, loss 0.498000, accuracy 0.827100
epoch 16, loss 0.491325, accuracy 0.831000
epoch 17, loss 0.488750, accuracy 0.831900
epoch 18, loss 0.488092, accuracy 0.830800
epoch 19, loss 0.484231, accuracy 0.833100
epoch 20, loss 0.482003, accuracy 0.832600
